In [16]:
import requests 
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import pickle
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from gensim.models import Word2Vec
import nltk

In [40]:
# 이쁘게 할 방법이... 어렵다...
def replacer(s):
    s = s.replace('\n', '').replace('\r', '').replace('\t', '')
    return s

In [4]:
# excel, pickle. db에 저장해야 하는뎅... ;ㅅ; 
def save(df, name):
    df.to_excel(name + ".xlsx")
    f = open(name + "_pickle.txt", 'w')
    pickle.dump(df, f)
    f.close()

In [17]:
BASE_URL = "http://store.steampowered.com"
CHART_BASE_URL = "http://steamcharts.com"
SPY_BASE_URL = "http://steamspy.com"

In [10]:
# 스팀차트에서 게임 리스트 크롤링 (세이브 해놓아서 이제 쓸일은 없는듯)
game_df = pd.DataFrame(columns=['name', 'link'])

for i in range(1, 10):
    steamchart_url = CHART_BASE_URL + "/top/p.{page_num}".format(page_num=i)
    response = requests.get(steamchart_url)
    dom = BeautifulSoup(response.content, 'html.parser')
    top_games_table = dom.select("table#top-games td.game-name.left")
    for top_game in top_games_table:
        name = replacer(top_game.select_one('a').text)
        link = top_game.select_one('a').attrs.get('href')
        game_df.loc[len(game_df)] = [name, link]

In [41]:
save(game_df, "game_df_old")

In [9]:
f = open("game_df_pickle.txt")
game_df = pickle.load(f)
f.close()

In [54]:
game_df

,name,link
0,Dota 2,/app/570
1,Counter-Strike: Global Offensive,/app/730
2,Team Fortress 2,/app/440
3,Sid Meier's Civilization V,/app/8930
4,Rust,/app/252490
5,ARK: Survival Evolved,/app/346110
6,Stardew Valley,/app/413150
7,Football Manager 2016,/app/378120
8,Garry's Mod,/app/4000
9,Rocket League,/app/252950


In [19]:
# 스팀차트에서 게임 플레이어 수 크롤링 (세이브 해놓아서 이제 쓸일은 없는듯2)
player_df = pd.DataFrame(columns=['last', 'median', 'mean'])

for i, row in game_df.iterrows():
    player_list = []
    steamchart_url = BASE_URL + row['link']
    response = requests.get(steamchart_url)
    dom = BeautifulSoup(response.content, 'html.parser')
    player_table = dom.select("div#content-wrapper td.right.num-f")
    for player in player_table:
        player_list.append(float(player.text))
    
    player_df.loc[len(player_df)] = [player_list[-1], np.median(player_list), np.mean(player_list)]


In [ ]:
save(player_df, "player_df")

In [27]:
f = open("player_df_pickle.txt")
player_df = pickle.load(f)
f.close()

In [63]:
driver = webdriver.Firefox()
weighted_tag_df = pd.DataFrame(columns=['name', 'tag', 'weight'])
game_tag_list = []
not_loaded_game = []
for i, row in game_df.iterrows():
    steam_url = BASE_URL + row['link']
    driver.get(steam_url)
    
    try:
        select_day = Select(driver.find_element_by_name('ageDay'))
        select_day.select_by_value('25')
        select_month = Select(driver.find_element_by_name('ageMonth'))
        select_month.select_by_value('February')
        select_year = Select(driver.find_element_by_name('ageYear'))
        select_year.select_by_value('1988')
        driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
    except NoSuchElementException:
        pass
    
    tag_elements = driver.find_elements_by_css_selector("div.glance_tags.popular_tags a")
    tag_list = []
    i = len(tag_elements)
    for tag in tag_elements:
        tag_text = driver.execute_script("return arguments[0].innerHTML", tag)
        if(tag_text is not u''):
            replaced_tag_text = replacer(tag_text)
            print(replaced_tag_text)
            weighted_tag_df.loc[len(weighted_tag_df)] = [row['name'], replaced_tag_text, i/len(tag_elements)]
            tag_list.append(replaced_tag_text)
            i -= 1
    if(tag_list is not []):
        game_tag_list.append(tag_list)
    else:
        not_loaded_game.append(row['name'])
        game_df = game_df[game_df['name'] != row['name']]
driver.quit()    

Free to Play
MOBA
Strategy
Multiplayer
Team-Based
Action
e-sports
Competitive
Online Co-Op
RTS
PvP
RPG
Tower Defense
Difficult
Fantasy
Co-op
Replay Value
Character Customization
Action RPG
Simulation
FPS
Multiplayer
Shooter
Action
Team-Based
Competitive
Tactical
First-Person
e-sports
PvP
Online Co-Op
Military
Co-op
Strategy
War
Trading
Realistic
Difficult
Fast-Paced
Moddable
Free to Play
Multiplayer
FPS
Action
Shooter
Class-Based
Trading
Funny
Team-Based
Cartoony
First-Person
Competitive
Co-op
Online Co-Op
Comedy
Robots
Tactical
Crafting
Cartoon
Moddable
Turn-Based Strategy
Strategy
Turn-Based
Multiplayer
Historical
4X
Singleplayer
Hex Grid
Grand Strategy
Co-op
Replay Value
Moddable
Tactical
Economy
Diplomacy
Simulation
Great Soundtrack
Touch-Friendly
Classic
Education

Survival
Crafting
Multiplayer
Open World
Sandbox
PvP
Adventure
Building
Indie
Action
FPS
First-Person
Nudity
Co-op
Shooter
Simulation
Online Co-Op
Post-apocalyptic
Survival Horror
RPG
Simulation
Indie
Agriculture
Pixel 

In [35]:
not_service_game = ['ARK: Survival Evolved',
'SMITE',
'Football Manager 2015',
'FINAL FANTASY XIV: A Realm Reborn',
'Path of Exile',
'Age of Empires II: HD Edition',
'Spacewar',
'Football Manager 2014',
'TERA',
'Counter-Strike Nexon: Zombies',
'Master of Orion',
'ArcheAge',
'Company of Heroes (New Steam Version)',
'Football Manager 2013',
'Football Manager 2012',
'Age of Mythology: Extended Edition',
'Rocksmith 2014',
'CABAL Online',
'MapleStory',
'FreeStyle2: Street Basketball',
'Dragon Age: Origins - Ultimate Edition']

In [64]:
game_tag_list

[[u'Free to Play',
  u'MOBA',
  u'Strategy',
  u'Multiplayer',
  u'Team-Based',
  u'Action',
  u'e-sports',
  u'Competitive',
  u'Online Co-Op',
  u'RTS',
  u'PvP',
  u'RPG',
  u'Tower Defense',
  u'Difficult',
  u'Fantasy',
  u'Co-op',
  u'Replay Value',
  u'Character Customization',
  u'Action RPG',
  u'Simulation'],
 [u'FPS',
  u'Multiplayer',
  u'Shooter',
  u'Action',
  u'Team-Based',
  u'Competitive',
  u'Tactical',
  u'First-Person',
  u'e-sports',
  u'PvP',
  u'Online Co-Op',
  u'Military',
  u'Co-op',
  u'Strategy',
  u'War',
  u'Trading',
  u'Realistic',
  u'Difficult',
  u'Fast-Paced',
  u'Moddable'],
 [u'Free to Play',
  u'Multiplayer',
  u'FPS',
  u'Action',
  u'Shooter',
  u'Class-Based',
  u'Trading',
  u'Funny',
  u'Team-Based',
  u'Cartoony',
  u'First-Person',
  u'Competitive',
  u'Co-op',
  u'Online Co-Op',
  u'Comedy',
  u'Robots',
  u'Tactical',
  u'Crafting',
  u'Cartoon',
  u'Moddable'],
 [u'Turn-Based Strategy',
  u'Strategy',
  u'Turn-Based',
  u'Multiplayer',


In [71]:
model = Word2Vec(game_tag_list, window=5, min_count=5, size=100)

In [72]:
print (model.syn0.shape)

(122L, 100L)


In [73]:
print (model[u'Strategy'])

[ 0.09203962  0.04510783  0.10730784 -0.06388637  0.04483759 -0.04783171
 -0.13938737  0.08750682 -0.06907009 -0.14695464 -0.07650381 -0.04949827
 -0.02643967  0.01267252 -0.130375    0.04487791 -0.0602975  -0.0261456
 -0.19665706  0.05134324 -0.07274029 -0.02233616 -0.1409606   0.2743223
 -0.09403681 -0.0432988   0.11476636  0.06123019 -0.05235332 -0.03411888
  0.11148679 -0.05974303  0.11401214  0.25221083 -0.2343884  -0.18571743
 -0.02766627 -0.10765514 -0.31933853  0.11458523  0.0633904  -0.02630725
 -0.16726591 -0.09121754 -0.0089567  -0.16095647  0.15206961  0.00952566
  0.06495027 -0.08878065  0.01962349  0.10105288  0.0398253  -0.11242212
  0.05627474 -0.07865559 -0.15072463 -0.11058834 -0.17187516  0.03310327
 -0.01310519 -0.11782493 -0.03914298 -0.04916824  0.0354878   0.0230182
 -0.09254121 -0.13632289  0.06471381  0.01411732 -0.09510358 -0.1225604
  0.04336205 -0.05908677 -0.13049965  0.17164274  0.0612965  -0.0157265
 -0.09528214  0.04123463 -0.16516349 -0.00354983  0.0934

In [77]:
model.most_similar(positive=[u'RPG'], topn=10)

[(u'Atmospheric', 0.9993453025817871),
 (u'Multiplayer', 0.999323308467865),
 (u'Action', 0.9993125200271606),
 (u'Simulation', 0.9992951154708862),
 (u'Singleplayer', 0.9992901086807251),
 (u'Online Co-Op', 0.9992725849151611),
 (u'Adventure', 0.9992412328720093),
 (u'Moddable', 0.9992401003837585),
 (u'Indie', 0.9992325305938721),
 (u'Great Soundtrack', 0.9992193579673767)]

In [78]:
model.most_similar(positive=['Open World','RPG'], negative=['Post-apocalyptic'], topn=10)

[(u'Sci-fi', 0.9974915981292725),
 (u'Online Co-Op', 0.9972966909408569),
 (u'Singleplayer', 0.9972550272941589),
 (u'Moddable', 0.9972319602966309),
 (u'Character Customization', 0.9972226023674011),
 (u'Great Soundtrack', 0.9971922039985657),
 (u'FPS', 0.9971805214881897),
 (u'Multiplayer', 0.9971762895584106),
 (u'Action', 0.997140645980835),
 (u'Atmospheric', 0.9971376061439514)]

In [79]:
model.most_similar(positive=['Open World','RPG'], topn=10)

[(u'Atmospheric', 0.9996486306190491),
 (u'Action', 0.9996254444122314),
 (u'Multiplayer', 0.9996200799942017),
 (u'Singleplayer', 0.9996023774147034),
 (u'Adventure', 0.9995899796485901),
 (u'Simulation', 0.9995710253715515),
 (u'Online Co-Op', 0.9995598793029785),
 (u'Great Soundtrack', 0.9995583295822144),
 (u'Indie', 0.999485433101654),
 (u'Moddable', 0.9994834065437317)]

In [ ]:
model.most_similar(positive=['Open World','RPG'], topn=10)

In [65]:
not_loaded_game

[]

In [66]:
game_df

,name,link
0,Dota 2,/app/570
1,Counter-Strike: Global Offensive,/app/730
2,Team Fortress 2,/app/440
3,Sid Meier's Civilization V,/app/8930
4,Rust,/app/252490
6,Stardew Valley,/app/413150
7,Football Manager 2016,/app/378120
8,Garry's Mod,/app/4000
9,Rocket League,/app/252950
10,Fallout 4,/app/377160


In [103]:
weighted_tag_df.ix[weighted_tag_df['tag'] == u'','tag'] = u'Early Access'

In [107]:
weighted_tag_df.ix[weighted_tag_df['tag'] == u'Early Access','name']

80                              Rust
325                         Unturned
385           H1Z1: King of the Kill
513                         Factorio
573                        Robocraft
633               H1Z1: Just Survive
653                             DayZ
853                       Brawlhalla
1033               Heroes & Generals
1108                   7 Days to Die
1188                 Space Engineers
1256      Don't Starve Together Beta
1588                  Portal Knights
1638                       Starbound
1658                      The Forest
1718                    Naval Action
1812                      Subnautica
2005                     Creativerse
2203                 Killing Floor 2
2323            Dungeon Defenders II
2557                       Hurtworld
2577                         Nosgoth
2597                 ShellShock Live
2806                  Scrap Mechanic
3013                           Squad
3195                    BeamNG.drive
3351                   Slime Rancher
3

In [108]:
save(weighted_tag_df, 'weighted_tag_df')

In [101]:
f = open("weighted_tag_df_pickle.txt")
weighted_tag_df = pickle.load(f)
f.close()

In [141]:
X = pd.DataFrame(columns=weighted_tag_df["tag"].unique())

In [142]:
X["name"] = game_df["name"]

In [143]:
for i, row in weighted_tag_df.iterrows():
    X[row["tag"]][X["name"] == row["name"]] = row["weight"]

In [144]:
X = X.fillna(0.)

In [145]:
X.describe().T.sort_values("mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max
Multiplayer,204,0.542289,0.332458,0,0.25,0.650000,0.805882,1.000000
Action,204,0.517789,0.355913,0,0.20,0.577778,0.850000,1.000000
Singleplayer,204,0.364671,0.285099,0,0.10,0.300000,0.643908,0.900000
Adventure,204,0.320351,0.319938,0,0.00,0.232143,0.626645,1.000000
Open World,204,0.319224,0.381131,0,0.00,0.050000,0.700000,1.000000
Strategy,204,0.313403,0.378178,0,0.00,0.125000,0.600000,1.000000
Co-op,204,0.269331,0.303782,0,0.00,0.150000,0.500000,1.000000
RPG,204,0.250083,0.362918,0,0.00,0.000000,0.462500,1.000000
Simulation,204,0.241793,0.351960,0,0.00,0.000000,0.500000,1.000000
Free to Play,204,0.237843,0.414492,0,0.00,0.000000,0.200000,1.000000


In [73]:
save(X, 'X')

In [30]:
f = open("X_pickle.txt")
X = pickle.load(f)
f.close()

In [139]:
for row in X.itertuples(index=False):
    print(sum(row[:-1]))

10.5
10.5
10.5
10.5
10.5
10.5
5.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.0
8.5
10.5
10.5
10.5
10.5
4.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
8.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
4.5
10.5
10.5
6.5
10.5
10.5
10.5
6.0
10.5
10.5
9.0
10.5
10.5
10.5
10.5
5.5
1.5
10.5
10.5
10.5
7.5
10.5
8.5
10.5
10.5
10.5
10.5
7.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
9.0
10.5
5.5
7.0
10.5
8.5
9.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
9.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
6.5
10.5
2.5
10.5
9.5
10.5
10.5
10.5
10.5
10.5
10.5
5.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
9.5
5.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
8.5
10.5
10.0
4.0
10.5
10.5
10.5
8.5
10.5
6.0
1.0
10.5
8.0
7.5
10.5
10.0
10.5
5.5
10.5
8.5
10.5
10.5
8.0
10.5
5.5
10.5
10.5
7.5
9.0
10.5
10.5
10.5
10.5
10.5
8.0
10.5
10.5
9.5
5.0
10.5


In [7]:
driver = webdriver.Firefox()
game_top_review_list = pd.DataFrame(columns=["game_name", "sentence"])
for i, row in game_df.iterrows():
    steam_url = BASE_URL + row["link"]
    driver.get(steam_url)
    
    try:
        select_day = Select(driver.find_element_by_name('ageDay'))
        select_day.select_by_value('25')
        select_month = Select(driver.find_element_by_name('ageMonth'))
        select_month.select_by_value('February')
        select_year = Select(driver.find_element_by_name('ageYear'))
        select_year.select_by_value('1988')
        driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
    except NoSuchElementException:
        pass
    
    review_elements = driver.find_elements_by_css_selector("div#Reviewsall0 div.content")
    review_sentence_list = []
    for review in review_elements:
        if(review.get_attribute('innerHTML') != u''):
            sentence = []
            for i, row in review.get_attribute('innerHTML'):
                words = str(row).split()
                sentence.append( words )
            review_sentence_list.append(sentence)
driver.quit()   

In [13]:
review_sentence_test = review_test[0].find_all(text=True)

In [21]:
replacer(review_sentence_test[-1]) == u''

True

In [147]:
driver = webdriver.Firefox()

driver.get("http://store.steampowered.com/app/252490/")

try:
    select_day = Select(driver.find_element_by_name('ageDay'))
    select_day.select_by_value('25')
    select_month = Select(driver.find_element_by_name('ageMonth'))
    select_month.select_by_value('February')
    select_year = Select(driver.find_element_by_name('ageYear'))
    select_year.select_by_value('1988')
    driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
except NoSuchElementException:
    pass

review_elements = driver.find_elements_by_css_selector("div#Reviewsall0 div.content")
review_elements[0].get_attribute('innerHTML')
#review_sentence_list = []
#for review in review_elements:
#    if(review.get_attribute('innerHTML') != u''):
#        sentence = []
#        for i, row in review.get_attribute('innerHTML'):
#            words = str(row).split()
#            sentence.append( words )
#        review_sentence_list.append(sentence)

driver.quit()

<selenium.webdriver.remote.webelement.WebElement (session="bb19dd26-8652-49f2-b6f1-062a4e07749f", element="{f75489c4-adb2-4d4f-b25e-6242824e26b4}")>

In [164]:
review_elements[0].text.split()

[u'I',
 u'start.',
 u'Spawn',
 u'in.',
 u'Nude,',
 u'cold,',
 u'helpless.',
 u'I',
 u'come',
 u'to',
 u'a',
 u'castle',
 u'looking',
 u'structure',
 u'(obviously',
 u'player',
 u'made)',
 u'I',
 u'go',
 u'up',
 u'to',
 u'it',
 u'and',
 u'say',
 u'the',
 u'oblivious',
 u'"Hello!?',
 u"I'm",
 u'friendly"',
 u'Meet',
 u'some',
 u'friendly',
 u'guys',
 u'out',
 u'of',
 u'the',
 u'castle',
 u'They',
 u'inject',
 u'me',
 u'with',
 u'a',
 u'green',
 u'and',
 u'white',
 u'pen',
 u'looking',
 u'thing',
 u'Puts',
 u'my',
 u'health',
 u'to',
 u'100',
 u'I',
 u'get',
 u'hit',
 u'by',
 u'a',
 u'morningstar',
 u'(or',
 u'a',
 u'mace',
 u'if',
 u'you',
 u'want',
 u'me',
 u'to',
 u'be',
 u'correct)',
 u'100HP>20HP',
 u'"You',
 u'are',
 u'our',
 u'slave',
 u'now"',
 u'I',
 u'accept',
 u'my',
 u'duty,',
 u'as',
 u'if',
 u'I',
 u'leave,',
 u'I',
 u'am',
 u'sure',
 u'of',
 u'my',
 u'death',
 u'They',
 u'show',
 u'me',
 u'the',
 u'contents',
 u'of',
 u'a',
 u'box',
 u'outside',
 u'of',
 u'the',
 u'"slave',

In [165]:
driver.quit()  

In [38]:
driver = webdriver.Firefox()
reviewer_link_df = pd.DataFrame(columns=['name', 'reviewer_link'])
reviewer_link_list = []
for i, row in game_df.iterrows():
    steam_url = BASE_URL + row['link']
    driver.get(steam_url)
    
    try:
        select_day = Select(driver.find_element_by_name('ageDay'))
        select_day.select_by_value('25')
        select_month = Select(driver.find_element_by_name('ageMonth'))
        select_month.select_by_value('February')
        select_year = Select(driver.find_element_by_name('ageYear'))
        select_year.select_by_value('1988')
        driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
    except NoSuchElementException:
        pass

    onegame_reviewer_link_list = []
    reviewer_link_elements = driver.find_elements_by_css_selector("div#Reviewsall0 div.num_owned_games a")
    for reviewer_link_element in reviewer_link_elements:
        reviewer_link = reviewer_link_element.get_attribute('href')
        print(reviewer_link)
        onegame_reviewer_list.append(reviewer_link)
        reviewer_link_df.loc[len(reviewer_link_df)] = [row['name'], reviewer_link]
        #print(reviewer.text)
    reviewer_link_list.append(onegame_reviewer_link_list)
driver.quit()

http://steamcommunity.com/id/revyhuang/games/?tab=all
http://steamcommunity.com/id/mikewell/games/?tab=all
http://steamcommunity.com/profiles/76561198245178862/games/?tab=all
http://steamcommunity.com/id/eternitydia/games/?tab=all
http://steamcommunity.com/profiles/76561198120907290/games/?tab=all
http://steamcommunity.com/id/DraganMKD87/games/?tab=all
http://steamcommunity.com/id/IsabellaDs/games/?tab=all
http://steamcommunity.com/id/thisguyisdope/games/?tab=all
http://steamcommunity.com/id/Artaxeus/games/?tab=all
http://steamcommunity.com/id/CpJenkaz/games/?tab=all
http://steamcommunity.com/id/iamseanliu/games/?tab=all
http://steamcommunity.com/id/okiharajones/games/?tab=all
http://steamcommunity.com/id/wildpiggie/games/?tab=all
http://steamcommunity.com/id/ServerNotFound/games/?tab=all
http://steamcommunity.com/id/TurnedVeggie/games/?tab=all
http://steamcommunity.com/profiles/76561198011645486/games/?tab=all
http://steamcommunity.com/id/appleducks/games/?tab=all
http://steamcommunit

''reviewer_link_df')' was not found in history, as a file, url, nor in the user namespace.


In [20]:
driver = webdriver.Firefox()
driver.get("http://store.steampowered.com/app/730/")
reviewer_elements = driver.find_elements_by_css_selector("div#Reviewsall0 div.num_owned_games a")
for reviewer_element in reviewer_elements:
    reviewer = reviewer_element.get_attribute('href')
    print(reviewer)
driver.quit()

http://steamcommunity.com/id/DraganMKD87/games/?tab=all
http://steamcommunity.com/id/IsabellaDs/games/?tab=all
http://steamcommunity.com/id/thisguyisdope/games/?tab=all
http://steamcommunity.com/id/Artaxeus/games/?tab=all
http://steamcommunity.com/id/CpJenkaz/games/?tab=all


In [ ]:
#BASE_URL = "http://steamcommunity.com/id/{username}/games/?tab=all"

owner_df = pd.DataFrame(columns=['name', 'game'])
owner_list = []

for name in owner_test:
    owner_url = BASE_URL.format(username=name)
    response = requests.get(owner_url)
    dom = BeautifulSoup(response.content, 'html.parser')
    games_table = dom.select("div#games_list_rows div.gameListRow")
    for game in games_table:
        name = replacer(top_game.select_one('a').text)
        link = top_game.select_one('a').attrs.get('href')
        game_df.loc[len(game_df)] = [name, link]

In [2]:
#BASE_URL = "http://steamcommunity.com/id/RevyHuang/games/?tab=all"

owner_df = pd.DataFrame(columns=['name', 'game'])
owner_list = []



In [4]:
driver = webdriver.Firefox()
driver.get("http://steamcommunity.com/id/RevyHuang/games/?tab=all")
game_elements = driver.find_elements_by_css_selector("div#games_list_rows div.gameListRowItemName.ellipsis ")

In [5]:
game_elements[0].text

u'Dota 2'

In [7]:
for game in game_elements:
    print(game.text)

Dota 2
PAYDAY 2
The Elder Scrolls V: Skyrim
Counter-Strike: Global Offensive
Left 4 Dead 2
Tom Clancy's Ghost Recon Phantoms - NA
Don't Starve Together Beta
Unturned
L.A. Noire
Don't Starve
Hitman: Absolution
How to Survive
Nosgoth
Borderlands 2
Resident Evil 6 / Biohazard 6
Dirty Bomb
Sleeping Dogs: Definitive Edition
Trine 2
MURDERED: SOUL SUSPECT™
Saints Row IV
Saints Row: The Third
ARK: Survival Evolved
Castle Crashers
Portal 2
Yesterday
Happy Wars
Trine
Warframe
Mortal Kombat Komplete Edition
Injustice: Gods Among Us Ultimate Edition
The Binding of Isaac
Hitman: Blood Money
Ascend: Hand of Kul
Portal
Sonic & All-Stars Racing Transformed
RIFT
Panzar
Dead Island
Garry's Mod
Dungeon Defenders II
Grand Theft Auto V
The Vanishing of Ethan Carter
Octodad: Dadliest Catch
Hotline Miami
Thief
Hitman 2: Silent Assassin
Strife
Lara Croft and the Guardian of Light
BattleBlock Theater
AdVenture Capitalist
Team Fortress 2
Emily is Away
Goat Simulator
Brothers - A Tale of Two Sons
Chivalry: Medi

In [8]:
driver.quit()  